In [1]:
import math

from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import scipy
import statsmodels.formula.api as smf
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')

In [2]:
# Read data using pandas
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls')

# Delete first three rows
df = df.drop([0,1,2], axis=0)

# Make first row the column headers
df = df.reset_index(drop=True)
df.columns = df.iloc[0]
df = df.drop([0], axis=0)
df = df.reset_index(drop=True)

# Rename all column headers
df.columns = ['City', 'Population', 'Violent Crime', 'Murder and Nonnegligent Manslaughter', 'Rape (revised definition)', 'Rape (legacy definition)', 'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft', 'Arson']

# Remove entire Unnamed: 4 or 'Rape revised defintion'
df = df.drop('Rape (revised definition)', axis=1)

# Remove null objects
df = df.drop([348, 349, 350], axis=0)
df = df.drop('Arson', axis=1)
df = df.dropna(how='all')

# Convert all columns from object to integer values
df[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']] = df[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']].astype(int)

# Remove New York and buffalo from data because it skews the data. Although the data is correct
df[df['City'] == 'New York']
df = df[(df['City']!='New York')&(df['City']!='Buffalo')]

# Change name of Property Crime so that it's one word
df['Propertycrime'] = df['Property Crime']
df = df.drop(columns=['Property Crime'])

df['Murder'] = df['Murder and Nonnegligent Manslaughter']
df = df.drop(columns=['Murder and Nonnegligent Manslaughter'])

df['Aggravated_Assault'] = df['Aggravated Assault']
df = df.drop(columns=['Aggravated Assault'])

In [3]:
df.head()

,City,Population,Violent Crime,Rape (legacy definition),Robbery,Burglary,Larceny-Theft,Motor Vehicle Theft,Propertycrime,Murder,Aggravated_Assault
0,Adams Village,1861,0,0,0,2,10,0,12,0,0
1,Addison Town and Village,2577,3,0,0,3,20,1,24,0,3
2,Akron Village,2846,3,0,0,1,15,0,16,0,3
3,Albany,97956,791,30,227,705,3243,142,4090,8,526
4,Albion Village,6388,23,3,4,53,165,5,223,0,16


In [4]:
# Create Features: Population^2, Murder binary, Robbery, binary
# More specifically, create binary categories for murder and robery
df['Population^2'] = df['Population']**2
df['Murder'] = np.where(df['Murder']>0, '1', '0')
df['Robbery_binary'] = np.where(df['Robbery']>0, '1', '0')
#df['Aggravated_Assault'] = np.where(df['Aggravated_Assault']>0, '1', '0')

#Convert new columns to int
df[['Population^2']] = df[['Population^2']].astype(int)
df[['Murder']] = df[['Murder']].astype(int)
df[['Robbery_binary']] = df[['Robbery_binary']].astype(int)
df[['Aggravated_Assault']] = df[['Aggravated_Assault']].astype(int)

In [5]:
# Run Model
regr = linear_model.LinearRegression()
y = df['Propertycrime'].values.reshape(-1, 1)
x = df[['Population','Population^2', 'Murder', 'Robbery_binary']]
regr.fit(x,y)

# Inspect the results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:\n', regr.score(x,y))


Coefficients: 
 [[1.74484226e-02 7.28298619e-08 1.84108347e+02 6.20590854e+01]]

Intercept: 
 [-35.45330011]

R-squared:
 0.7572767492306005


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [6]:
# Cross Validation test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)
print('With 30% holdout: ' + str(regr.fit(x_train, y_train,).score(x_test, y_test)))
print('Testing on sample:' + str(regr.fit(x,y).score(x,y)))

With 30% holdout: 0.7503983756164383
Testing on sample:0.7572767492306005


In [7]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(regr, x, y, cv=5)

array([0.81409112, 0.71966237, 0.74405329, 0.61140936, 0.21822869])

In [8]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ Population+Population^2+Murder+Robbery_binary'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df).fit()

In [9]:
lm.params

Intercept        -80.074
Population         8.359
Population ^ 2    -8.330
Murder           114.402
Robbery_binary   -27.541
dtype: float64

In [10]:
lm.pvalues

Intercept        0.025
Population       0.456
Population ^ 2   0.458
Murder           0.135
Robbery_binary   0.578
dtype: float64

In [11]:
# Create PCA on the follwing variables: Burglary, Violent Crime, Aggravated_Assault, Larceny-Theft, Motor-Vehicle Theft, Robbery

# Standardize Data
features = ['Burglary', 'Violent Crime', 'Aggravated_Assault', 'Larceny-Theft']
x = df.loc[:, features].values

scaler = StandardScaler()

scaler.fit(x)

pd.DataFrame(data=x, columns=features).head()

# PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)

# Create Merged Data Frame
principalDf = pd.DataFrame(data = principalComponents, columns = ['principalcomponent_1', 'principalcomponent_2'])
principalDf['City'] = df['City']
merged_inner = pd.merge(left=df, right=principalDf, left_on='City', right_on='City')
df = merged_inner

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
# Run Model
regr = linear_model.LinearRegression()
y = df['Propertycrime'].values.reshape(-1, 1)
x = df[['Population','Population^2', 'Murder', 'Robbery_binary', 'principalcomponent_1', 'principalcomponent_2']]
regr.fit(x,y)

# Inspect the results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:\n', regr.score(x,y))


Coefficients: 
 [[1.04847039e-02 1.70042096e-07 1.90643236e+02 9.79970918e+01
  1.52002213e-02 6.42797921e-02]]

Intercept: 
 [-8.23669611]

R-squared:
 0.8831188362844727


In [13]:
# Cross Validation test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)
print('With 30% holdout: ' + str(regr.fit(x_train, y_train,).score(x_test, y_test)))
print('Testing on sample:' + str(regr.fit(x,y).score(x,y)))

With 30% holdout: 0.94577136316751
Testing on sample:0.8831188362844727


In [14]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(regr, x, y, cv=5)

array([0.78210715, 0.73976958, 0.70814505, 0.91106597, 0.87308833])

In [15]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ Population+Population^2+Murder+Robbery_binary+principalcomponent_1+principalcomponent_2'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df).fit()

In [16]:
lm.params

Intercept              -101.600
Population               11.433
Population ^ 2          -11.400
Murder                   57.394
Robbery_binary          -78.160
principalcomponent_1      0.001
principalcomponent_2      0.157
dtype: float64

In [17]:
lm.pvalues

Intercept              0.001
Population             0.246
Population ^ 2         0.248
Murder                 0.394
Robbery_binary         0.075
principalcomponent_1   0.982
principalcomponent_2   0.438
dtype: float64

In [18]:
# Process Second Dataset on Illinois

df1 = pd.read_excel('table_8_offenses_known_to_law_enforcement_illinois_by_city_2013.xls')
df1.tail()

,Table 8,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
509,Zion,24338,146,5,NaN,19,34,88,800,243,520,37,5
510,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
511,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
512,3 The FBI determined that the agency's data we...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
513,4 The data collection methodology for the offe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Delete first three rows
df1 = df1.drop([0,1,2], axis=0)

# Make first row the column headers
df1 = df1.reset_index(drop=True)
df1.columns = df1.iloc[0]
df1 = df1.drop([0], axis=0)
df1 = df1.reset_index(drop=True)

# Rename all column headers
df1.columns = ['City', 'Population', 'Violent Crime', 'Murder and Nonnegligent Manslaughter', 'Rape (revised definition)', 'Rape (legacy definition)', 'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary', 'Larceny-Theft', 'Motor Vehicle Theft', 'Arson']

# Remove entire Unnamed: 4 or 'Rape revised defintion'
df1 = df1.drop('Rape (revised definition)', axis=1)

# Remove null objects
df1 = df1.drop([77, 398, 506, 507, 508, 509], axis=0)
df1 = df1.drop('Arson', axis=1)
df1 = df1.dropna(how='all')

# Convert all columns from object to integer values
df1[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']] = df1[['Population', 'Violent Crime',
       'Murder and Nonnegligent Manslaughter', 'Rape (legacy definition)',
       'Robbery', 'Aggravated Assault', 'Property Crime', 'Burglary',
       'Larceny-Theft', 'Motor Vehicle Theft']].astype(int)

# Remove New York and buffalo from data because it skews the data. Although the data is correct
#df1[df1['City'] == 'New York']
#df1 = df1[(df1['City']!='New York')&(df1['City']!='Buffalo')]

# Change name of Property Crime so that it's one word
df1['Propertycrime'] = df1['Property Crime']
df1 = df1.drop(columns=['Property Crime'])

df1['Murder'] = df1['Murder and Nonnegligent Manslaughter']
df1 = df1.drop(columns=['Murder and Nonnegligent Manslaughter'])

df1['Aggravated_Assault'] = df1['Aggravated Assault']
df1 = df1.drop(columns=['Aggravated Assault'])

In [20]:
# Create Features: Population^2, Murder binary, Robbery, binary
# More specifically, create binary categories for murder and robery
df1['Population^2'] = df1['Population']**2
df1['Murder'] = np.where(df1['Murder']>0, '1', '0')
df1['Robbery_binary'] = np.where(df1['Robbery']>0, '1', '0')
#df1['Aggravated_Assault'] = np.where(df1['Aggravated_Assault']>0, '1', '0')

#Convert new columns to int
df1[['Population^2']] = df1[['Population^2']].astype(int)
df1[['Murder']] = df1[['Murder']].astype(int)
df1[['Robbery_binary']] = df1[['Robbery_binary']].astype(int)
df1[['Aggravated_Assault']] = df1[['Aggravated_Assault']].astype(int)

In [21]:
# Create PCA on the follwing variables: Burglary, Violent Crime, Aggravated_Assault, Larceny-Theft, Motor-Vehicle Theft, Robbery

# Standardize Data
features = ['Burglary', 'Violent Crime', 'Aggravated_Assault', 'Larceny-Theft']
x = df1.loc[:, features].values

scaler = StandardScaler()

scaler.fit(x)

pd.DataFrame(data=x, columns=features).head()

# PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)

# Create Merged Data Frame
principalDf1 = pd.DataFrame(data = principalComponents, columns = ['principalcomponent_1', 'principalcomponent_2'])
principalDf1['City'] = df1['City']
merged_inner = pd.merge(left=df1, right=principalDf1, left_on='City', right_on='City')
df1 = merged_inner

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [22]:
# Run Model
y = df1['Propertycrime'].values.reshape(-1, 1)
x = df1[['Population','Population^2', 'Murder', 'Robbery_binary', 'principalcomponent_1', 'principalcomponent_2']]
regr.fit(x,y)

# Inspect the results
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:\n', regr.score(x,y))


Coefficients: 
 [[ 2.13451881e-02 -7.64984534e-09  8.55545067e+01  3.69410487e+01
  -2.23687624e-02 -2.31624078e-01]]

Intercept: 
 [-42.64251966]

R-squared:
 0.7354182862362844


In [23]:
# Cross Validation test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=30)
print('With 30% holdout: ' + str(regr.fit(x_train, y_train,).score(x_test, y_test)))
print('Testing on sample:' + str(regr.fit(x,y).score(x,y)))

With 30% holdout: 0.6926982267936467
Testing on sample:0.7354182862362844


In [24]:
# Cross-validation with mutliple folds
from sklearn.model_selection import cross_val_score

cross_val_score(regr, x, y, cv=5)

array([0.56639313, 0.68661144, 0.81365572, 0.71811264, 0.64732305])

In [25]:
# Test for significance in parameters
linear_formula = 'Propertycrime ~ Population+Population^2+Murder+Robbery_binary+principalcomponent_1+principalcomponent_2'

# Fit the model to our data using formula
lm = smf.ols(formula=linear_formula, data=df1).fit()

In [26]:
lm.params

Intercept              -39.595
Population              -3.513
Population ^ 2           3.534
Murder                  87.944
Robbery_binary          44.653
principalcomponent_1    -0.025
principalcomponent_2    -0.220
dtype: float64

In [27]:
lm.pvalues

Intercept              0.034
Population             0.569
Population ^ 2         0.566
Murder                 0.016
Robbery_binary         0.108
principalcomponent_1   0.400
principalcomponent_2   0.222
dtype: float64